## 1. Execute imports

In [4]:
import pandas as pd
import numpy as np
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import classification_report

In [5]:
import warnings
warnings.filterwarnings("ignore")
data= pd.read_csv("../formula-interpreter/glass.csv")
X = pd.read_csv('../formula-interpreter/glass_formula_a.csv', encoding = 'utf-8', delimiter=',')
y = data['Type']
features = list(X.columns.values)
y = pd.DataFrame(y)

## KNN

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from numpy import mean
from numpy import std

### Information gain

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'selector__k': ['all']},
                {'classifier': [KNeighborsClassifier()]}]

In [7]:
%%time
knn_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_info.fit(X_train, y_train)

Wall time: 4.09 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000001371F05CB80>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [KNeighborsClassifier()]}])

In [8]:
print(cross_val_score(knn_info, X, y, cv=10).mean())

0.6835782638414218


In [8]:
features = list(X.columns.values)

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', KNeighborsClassifier())])
    # define search space
    search_space = [{'selector__k': ['all']}, {'classifier': [KNeighborsClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f' % (acc, result.best_score_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

>acc=0.651, est=0.632
>acc=0.674, est=0.620
>acc=0.628, est=0.602
>acc=0.605, est=0.638
>acc=0.667, est=0.622
Accuracy: 0.645 (0.026)


### Variance threshold

In [10]:
from sklearn.feature_selection import VarianceThreshold

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'classifier': [KNeighborsClassifier()]}]

In [11]:
%%time
knn_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
knn_variance.fit(X_train, y_train)

Wall time: 110 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}],
             scoring='accuracy')

In [12]:
print(cross_val_score(knn_variance, X, y, cv=10).mean())

0.7187628161312372


In [11]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', KNeighborsClassifier())])

    search_space = [{'classifier': [KNeighborsClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = KNeighborsClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.698, est=0.615, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
>acc=0.721, est=0.621, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
>acc=0.628, est=0.609, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
>acc=0.605, est=0.627, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
>acc=0.714, est=0.594, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
Accuracy: 0.673 (0.048)
Wall time: 507 ms


### Chi-Square

In [12]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', KNeighborsClassifier())])

In [15]:
search_space = [{'classifier': [KNeighborsClassifier()]}]

In [16]:
%%time
knn_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_chi.fit(X_train, y_train)

Wall time: 131 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001371EE50040>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}])

In [17]:
print(cross_val_score(knn_chi, X, y, cv=10).mean())

0.6210526315789473


In [13]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', KNeighborsClassifier())])
    # define search space
    search_space = [{'classifier': [KNeighborsClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = KNeighborsClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.674, est=0.609, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', KNeighborsClassifier())])
>acc=0.698, est=0.597, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', KNeighborsClassifier())])
>acc=0.605, est=0.603, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', KNeighborsClassifier())])
>acc=0.581, est=0.621, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', KNeighborsClassifier())])
>acc=0.714, est=0.565, cfg=Pipeline(steps=[('sca

### Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

### Information gain

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', LogisticRegression())])

search_space = [{'selector__k': ['all']},
                {'classifier': [LogisticRegression()]}]

In [21]:
%%time
logistic_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_info.fit(X_train, y_train)

Wall time: 4.25 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000001371F05CB80>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [LogisticRegression()]}])

In [22]:
print(cross_val_score(logistic_info, X, y, cv=10).mean())

0.7748462064251538


In [15]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k=10)),
                 ('classifier', LogisticRegression())])
    # define search space
    search_space = [{'selector__k': [10]}, {'classifier': [LogisticRegression()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = LogisticRegression()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.581, est=0.603, cfg={'classifier': LogisticRegression()}
>acc=0.674, est=0.573, cfg={'classifier': LogisticRegression()}
>acc=0.628, est=0.597, cfg={'classifier': LogisticRegression()}
>acc=0.628, est=0.597, cfg={'classifier': LogisticRegression()}
>acc=0.714, est=0.621, cfg={'selector__k': 10}
Accuracy: 0.645 (0.045)
Wall time: 23 s


### Variance threshold

In [23]:
from sklearn.feature_selection import VarianceThreshold

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', LogisticRegression())])

search_space = [{'classifier': [LogisticRegression()]}]

In [25]:
%%time
logistic_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
logistic_variance.fit(X_train, y_train)

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

Wall time: 471 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}],
             scoring='accuracy')

In [26]:
print(cross_val_score(logistic_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

0.677118933697881


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

In [16]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', LogisticRegression())])

    search_space = [{'classifier': [LogisticRegression()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = LogisticRegression()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.744, est=0.656, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
>acc=0.698, est=0.591, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
>acc=0.721, est=0.621, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
>acc=0.628, est=0.609, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
>acc=0.714, est=0.588, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
Accuracy: 0.701 (0.039)
Wall time: 2.02 s


### Chi-Square

In [27]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', LogisticRegression())])

In [29]:
search_space = [{'classifier': [LogisticRegression()]}]

In [30]:
%%time
logistic_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_chi.fit(X_train, y_train)

Wall time: 182 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001371EE50040>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}])

In [31]:
print(cross_val_score(logistic_chi, X, y, cv=10).mean())

0.7578947368421053


In [17]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', LogisticRegression())])
    # define search space
    search_space = [{'classifier': [LogisticRegression()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = LogisticRegression()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.605, est=0.592, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', LogisticRegression())])
>acc=0.698, est=0.550, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', LogisticRegression())])
>acc=0.674, est=0.574, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', LogisticRegression())])
>acc=0.581, est=0.598, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', LogisticRegression())])
>acc=0.738, est=0.542, cfg=Pipeline(steps=[('scaler', Mi

## SVM

In [18]:
from sklearn.svm import SVC

### Information gain

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'selector__k': ['all']},
                {'classifier': [SVC(kernel='linear')]}]

In [35]:
%%time
svm_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_info.fit(X_train, y_train)

Wall time: 4.26 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000001371F05CB80>)),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [SVC(kernel='linear')]}])

In [36]:
print(cross_val_score(svm_info, X, y, cv=10).mean())

0.7630895420369106


In [19]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k=9)),
                 ('classifier', SVC(kernel='linear'))])
    # define search space
    search_space = [{'selector__k': [9]}, {'classifier': [SVC(kernel='linear')]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = SVC(kernel='linear')
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.628, est=0.644, cfg={'selector__k': 9}
>acc=0.721, est=0.585, cfg={'classifier': SVC(kernel='linear')}
>acc=0.651, est=0.544, cfg={'classifier': SVC(kernel='linear')}
>acc=0.651, est=0.609, cfg={'classifier': SVC(kernel='linear')}
>acc=0.690, est=0.639, cfg={'selector__k': 9}
Accuracy: 0.668 (0.033)
Wall time: 20.5 s


### Variance threshold

In [37]:
from sklearn.feature_selection import VarianceThreshold

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'classifier': [SVC(kernel='linear')]}]

In [39]:
%%time
svm_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
svm_variance.fit(X_train, y_train)

Wall time: 59.6 s


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'classifier': [SVC(kernel='linear')]}],
             scoring='accuracy')

In [40]:
print(cross_val_score(svm_variance, X, y, cv=10).mean())

0.7630895420369106


In [20]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold(3)),
                 ('classifier', SVC(kernel='linear'))])

    search_space = [{'classifier': [SVC(kernel='linear')]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = SVC(kernel='linear')
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

ValueError: No feature in X meets the variance threshold 3.00000

### Chi-Square

In [41]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', SVC())])

In [43]:
search_space = [{'classifier': [SVC()]}]

In [44]:
%%time
svm_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_chi.fit(X_train, y_train)

Wall time: 200 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001371EE50040>)),
                                       ('classifier', SVC())]),
             param_grid=[{'classifier': [SVC()]}])

In [45]:
print(cross_val_score(svm_chi, X, y, cv=10).mean())

0.7409603554340396


In [21]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', SVC(kernel='linear'))])
    # define search space
    search_space = [{'classifier': [SVC(kernel='linear')]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = SVC(kernel='linear')
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.628, est=0.539, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.674, est=0.427, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.721, est=0.527, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.651, est=0.533, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.786, est=0.494, cfg=Pipeline(steps=[('scaler', Mi

## Naive Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB

### Information gain

In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', GaussianNB())])

search_space = [{'selector__k': ['all']},
                {'classifier': [GaussianNB()]}]

In [49]:
%%time
nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)

Wall time: 3.3 s


GridSearchCV(estimator=Pipeline(steps=[('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000001371F05CB80>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [GaussianNB()]}])

In [50]:
print(cross_val_score(nb_info, X, y, cv=10).mean())

0.7552802460697198


In [23]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k=10)),
                 ('classifier', GaussianNB())])
    # define search space
    search_space = [{'selector__k': [10]}, {'classifier': [GaussianNB()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = GaussianNB()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.302, est=0.368, cfg={'selector__k': 10}
>acc=0.465, est=0.350, cfg={'classifier': GaussianNB()}
>acc=0.488, est=0.345, cfg={'classifier': GaussianNB()}
>acc=0.442, est=0.404, cfg={'selector__k': 10}
>acc=0.429, est=0.425, cfg={'classifier': GaussianNB()}
Accuracy: 0.425 (0.065)
Wall time: 25.2 s


### Variance threshold

In [51]:
from sklearn.feature_selection import VarianceThreshold

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', GaussianNB())])

search_space = [{'classifier': [GaussianNB()]}]

In [53]:
nb_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
nb_variance.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}], scoring='accuracy')

In [54]:
print(cross_val_score(nb_variance, X, y, cv=10).mean())

0.7552802460697198


In [24]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', GaussianNB())])

    search_space = [{'classifier': [GaussianNB()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = GaussianNB()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.419, est=0.473, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
>acc=0.558, est=0.420, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
>acc=0.581, est=0.479, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
>acc=0.419, est=0.421, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
>acc=0.429, est=0.472, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
Accuracy: 0.481 (0.073)
Wall time: 382 ms


### Chi-Square

In [55]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', GaussianNB())])

In [57]:
search_space = [{'classifier': [GaussianNB()]}]

In [58]:
%%time
nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

Wall time: 78.8 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001371EE50040>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}])

In [59]:
print(cross_val_score(nb_chi, X, y, cv=10).mean())

0.7552802460697198


In [25]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', GaussianNB())])
    # define search space
    search_space = [{'classifier': [GaussianNB()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = GaussianNB()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.279, est=0.409, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', GaussianNB())])
>acc=0.442, est=0.391, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', GaussianNB())])
>acc=0.442, est=0.373, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', GaussianNB())])
>acc=0.419, est=0.415, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', GaussianNB())])
>acc=0.381, est=0.401, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                (

In [5]:
## Random Forest

from sklearn.ensemble import RandomForestRegressor

### Information gain

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', RandomForestRegressor())])

search_space = [{'selector__k': ['all']},
                {'classifier': [RandomForestRegressor()]}]

nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)
print('Random Forest -> Information gain')
print(cross_val_score(nb_info, X, y, cv=10).mean())

### Chi-Square

from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', RandomForestRegressor())])

search_space = [{'classifier': [RandomForestRegressor()]}]

nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

print('Random Forest -> Chi-Square')
print(cross_val_score(nb_chi, X, y, cv=10).mean())

Random Forest -> Information gain
0.28540432711039365
Random Forest -> Chi-Square
0.2744094846507973


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier -> Info


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

...

MLPClassifier -> Chi


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

0.7499658236500342


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [26]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', MLPClassifier())])

search_space = [{'selector__k': ['all']},
                {'classifier': [MLPClassifier()]}]

nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)

print('MLPClassifier -> Info')
print(cross_val_score(nb_info, X, y, cv=10).mean())

In [27]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k=10)),
                 ('classifier', MLPClassifier())])
    # define search space
    search_space = [{'selector__k': [10]}, {'classifier': [MLPClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = MLPClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.628, est=0.627, cfg={'classifier': MLPClassifier()}
>acc=0.698, est=0.567, cfg={'selector__k': 10}
>acc=0.558, est=0.590, cfg={'selector__k': 10}
>acc=0.628, est=0.638, cfg={'selector__k': 10}
>acc=0.667, est=0.588, cfg={'classifier': MLPClassifier()}
Accuracy: 0.636 (0.047)
Wall time: 43.8 s


In [ ]:
### Variance threshold

from sklearn.feature_selection import VarianceThreshold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', VarianceThreshold()),
                 ('classifier', MLPClassifier())])

search_space = [{'classifier': [MLPClassifier()]}]

nb_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
nb_variance.fit(X_train, y_train)

print('MLPClassifier -> Variance')
print(cross_val_score(nb_variance, X, y, cv=10).mean())

In [28]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', MLPClassifier())])

    search_space = [{'classifier': [MLPClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = MLPClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.651, est=0.650, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
>acc=0.651, est=0.574, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
>acc=0.628, est=0.520, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
>acc=0.628, est=0.586, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
>acc=0.714, est=0.640, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
Accuracy: 0.654 (0.032)
Wall time: 12.2 s


In [ ]:
### Chi-Square

from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', MLPClassifier())])

search_space = [{'classifier': [MLPClassifier()]}]

nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

print('MLPClassifier -> Chi')
print(cross_val_score(nb_chi, X, y, cv=10).mean())

In [29]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', MLPClassifier())])
    # define search space
    search_space = [{'classifier': [MLPClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = MLPClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.581, est=0.616, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', MLPClassifier())])
>acc=0.651, est=0.556, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', MLPClassifier())])
>acc=0.674, est=0.609, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', MLPClassifier())])
>acc=0.628, est=0.592, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001DE328AEF70>)),
                ('classifier', MLPClassifier())])
>acc=0.714, est=0.588, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
     